In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.ensemble import BaseEnsemble
from scipy.stats import mode
import numpy as np

In [ ]:
from rolling_lookahead_dt_pulp import rollo_oct_pulp

# was hiermit eben nicht geht ist, dass man auf Trainingsdaten trainiert (was einem das reine Modell geben sollte). Dabei werden aber leider gleichzeitig
# die Testdaten auf diesen Modell predicted
# Das Resultat ist also, dass man nicht andere Testdaten auf dem fertigen modell testen kann



class CustomTreeWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, depth=None, criterion='gini', test_data=None, target_label=None, features=None):
        self.depth = depth
        self.criterion = criterion
        self.test_data = test_data
        self.target_label = target_label
        self.features = features
    
    def fit(self, X, y):
        if self.test_data is None or self.target_label is None or self.features is None:
            raise ValueError("Test data, target label, and features must be set before calling fit")
        
        # Construct train DataFrame or array compatible with rollo_oct_pulp.run interface
        train_df = ... # From X, y possibly reconstruct with target labels
        # Call your custom run with both train and stored test data
        self.result_dict_ = rollo_oct_pulp.run(
            train=train_df,
            test=self.test_data,
            target_label=self.target_label,
            features=self.features,
            depth=self.depth,
            criterion=self.criterion
        )
        self.is_fitted_ = True
        return self
    
    def predict(self, X):
        check_is_fitted(self, 'is_fitted_')
        # Implement prediction based on result_dict_
        ...

In [ ]:
class CustomRandomForest(BaseEnsemble, ClassifierMixin):
    def __init__(self, base_estimator=None, n_estimators=10, max_depth=None, random_state=None):
        super().__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            random_state=random_state,
        )
        self.max_depth = max_depth

    def _validate_estimators(self):
        # This ensures base_estimator is cloned n_estimators times
        super()._validate_estimators()
        if self.base_estimator is None:
            self.base_estimator = CustomTreeClassifier(max_depth=self.max_depth)

    def fit(self, X, y):
        self._validate_estimators()
        self.estimators_ = []
        n_samples = X.shape[0]
        rng = np.random.RandomState(self.random_state)

        for i in range(self.n_estimators):
            # Bootstrap sample indices
            indices = rng.choice(n_samples, size=n_samples, replace=True)
            X_sample, y_sample = X[indices], y[indices]
            estimator = clone(self.base_estimator)
            estimator.fit(X_sample, y_sample)
            self.estimators_.append(estimator)
        return self

    def predict(self, X):
        # Gather predictions from all trees
        predictions = np.array([tree.predict(X) for tree in self.estimators_])
        maj_vote, _ = mode(predictions, axis=0)
        return maj_vote.ravel()